## Cloning the repo

In [ ]:
!git clone https://github.com/Dineswar11/dretino

### **Installing the package**

In [ ]:
import os
os.chdir('./dretino')
!pwd
!pip install .

In [ ]:
!pip install -qU pytorch-lightning wandb timm
!pip install -qU torchmetrics --upgrade

## Importing Libraries

In [ ]:
import wandb
import pandas as pd

from sklearn.model_selection import train_test_split

import albumentations as A
from albumentations.pytorch import ToTensorV2


from dretino.dataloader.build_features import DRDataModule
from dretino.models.predict_model import test
from dretino.models.train_model import Model, train
from dretino.visualization.visualize import show_images, cal_mean, plot_metrics

## WandB Login

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
WANDB = user_secrets.get_secret("wandb")

wandb.login(key=WANDB)

## Reading the Dataset

In [ ]:
os.chdir('/kaggle/working')

PATH = '../input/indian-retina-classification/B._20Disease_20Grading/B. Disease Grading/'
IMG_PATH = '../input/indian-retina-classification/B._20Disease_20Grading/B. Disease Grading/1. Original Images/'
dfx = pd.read_csv(PATH+'2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv',usecols = ['Image name','Retinopathy grade'])
df_test = pd.read_csv(PATH+'2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv',usecols = ['Image name','Retinopathy grade'])
df_train, df_valid = train_test_split(
    dfx, 
    test_size=0.2, 
    random_state=42, 
    stratify=dfx['Retinopathy grade'].values
)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

df_train.head()

## Creating a DataModule

In [ ]:
train_transforms = A.Compose(
    [
        A.Resize(width=250, height=250),
        A.RandomCrop(height=224, width=224),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Blur(p=0.15),
        A.CLAHE(p=0.15),
        A.ColorJitter(p=0.15),
        A.Affine(shear=30, rotate=0, p=0.1),
        A.Normalize(
            mean=(0.5237, 0.2542, 0.0853),
            std=(0.2649, 0.1497, 0.0876),
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

val_transforms = A.Compose(
    [
        A.Resize(height=224, width=224),
        A.Normalize(
            mean=(0.5237, 0.2542, 0.0853),
            std=(0.2649, 0.1497, 0.0876),
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

test_transforms = A.Compose(
    [
        A.Resize(height=224, width=224),
        A.Normalize(
            mean=(0.5237, 0.2542, 0.0853),
            std=(0.2649, 0.1497, 0.0876),
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

dm = DRDataModule(df_train, df_valid, df_test,
                  train_path=IMG_PATH + 'images_resized',
                  valid_path=IMG_PATH + 'images_resized',
                  test_path=IMG_PATH + 'test_images_resized',
                  train_transforms=train_transforms,
                  val_transforms=val_transforms,
                  test_transforms=test_transforms,
                  num_workers=2,
                  batch_size=16)

#### Train Images

In [ ]:
show_images(dm.train_dataloader())

#### Validation Images

In [ ]:
show_images(dm.val_dataloader())

#### Test Images

In [ ]:
show_images(dm.test_dataloader())

## Training on resnet50d **No Additional Layers with Aug**

***Coral loss 200 epochs***

In [ ]:
args = dict(
    model_name='resnet50d',
    num_neurons=512,
    num_layers=2,
    dropout=0.2,
    lr=3e-4,
    loss='coral',
    epochs=200,
    gpus=1,
    project='DRD',
    additional_layers=False
)

wab = True
fast_dev_run = False
overfit_batches = False



file_name, trainer = train(Model, dm,
                           wab=wab,
                           fast_dev_run=fast_dev_run,
                           overfit_batches=overfit_batches,
                           **args)

print(file_name)

if not fast_dev_run:
    plot_metrics('../reports/csv_logs/' + file_name)

    test(Model, dm,
         file_name,
         trainer,
         wab=wab,
         fast_dev_run=fast_dev_run,
         overfit_batches=overfit_batches)